In [ ]:
#!/usr/bin/env python3
"""
Generate test brown-dwarf spectra varying Teff, g, fsed, and Kzz.

- TODAY: uses PICASO default opacity DB, your chosen R, and Sonora TP as-is.
- FUTURE: flip OPACITY_DB, RES_R, or replace/modify TP via a single hook.
- Clouds: auto-selected via Virga condensation curves (can be overridden).

Outputs: a single .npz with
  X = [Teff, g(m/s^2), fsed, Kzz(cm^2/s)]  shape (N, 4)
  Y = flux (regridded)                      shape (N, N_lambda)
  wavelength_um                             shape (N_lambda)
  meta_json (run configuration)
"""

def main():
    rng = np.random.default_rng(SEED)

    # Draw random cases
    Teff = rng_uniform_intv(rng, *TEFF_RANGE, size=N_SPECTRA)
    g_si = rng_uniform_intv(rng, *G_RANGE,   size=N_SPECTRA)
    fsed = rng_uniform_intv(rng, *FSED_RANGE, size=N_SPECTRA)
    kzz  = rng_log_uniform(rng, *KZZ_RANGE,  size=N_SPECTRA)

    # First run: lock wavelength grid
    bd0 = jdi.inputs(calculation="browndwarf")
    bd0.phase_angle(0)
    bd0.gravity(g_si[0], gravity_unit=u.Unit("m/s**2"))
    bd0.sonora(SONORA_DIR, Teff[0])

    # Base TP
    prof0 = bd0.inputs["atmosphere"]["profile"]
    P0 = np.asarray(prof0["pressure"], float)     # bars
    T0 = np.asarray(prof0["temperature"], float)  # K
    mmw0 = compute_mmw_from_profile(prof0)

    # TP hook (no-op today; future interpolation/pickle can go here)
    P0, T0 = tp_hook(P0, T0)

    # Kzz vector/scalar
    kz0 = kzz[0]
    if KZZ_IS_VECTOR:
        bd0.inputs["atmosphere"]["profile"]["kz"] = [float(kz0)] * len(P0)
    else:
        bd0.inputs["atmosphere"]["profile"]["kz"] = float(kz0)


    out0 = bd0.spectrum(opa, full_output=True)
    wn0, th0 = out0["wavenumber"], out0["thermal"]  # cm^-1, erg/cm^2/s/cm
    if RES_R is not None:
        wn0, th0 = jdi.mean_regrid(wn0, th0, R=int(RES_R))

    wl_um = 1e4 / wn0
    nw = wl_um.size

    # Allocate dataset
    X = np.empty((N_SPECTRA, 4), dtype=np.float32)
    Y = np.empty((N_SPECTRA, nw), dtype=np.float32)

    # Save first
    X[0] = [Teff[0], g_si[0], fsed[0], kzz[0]]
    Y[0] = th0.astype(np.float32)

    # Remaining runs
    for i in range(1, N_SPECTRA):


        # Sanity checks
        if np.any(~np.isfinite(th)) or np.any(th <= 0):
            print(f"[WARN] Skipping case i={i}: bad flux (NaN/<=0).")
            continue

        wl_um_i = 1e4 / wn
        if wl_um_i.shape != wl_um.shape or not np.allclose(wl_um_i, wl_um, rtol=0, atol=1e-10):
            raise RuntimeError("Wavelength grid changed—check PICASO version/regrid settings.")

        X[i] = [Teff[i], g_si[i], fsed[i], kzz[i]]
        Y[i] = th.astype(np.float32)

    meta = dict(
        sonora_dir=SONORA_DIR,
        virga_dir=VIRGA_DIR,
        opacity_db=OPACITY_DB,
        wave_range=WAVE_RANGE,
        R=RES_R,
        use_clouds=USE_CLOUDS,
        mh=MH,
        n_spectra=N_SPECTRA,
        seed=SEED,
        teff_range=TEFF_RANGE,
        g_range=G_RANGE,
        fsed_range=FSED_RANGE,
        kzz_range=KZZ_RANGE,
        clouds_override=CLOUDS_OVERRIDE,
        kzz_is_vector=KZZ_IS_VECTOR,
        tp_hook="no-op (ready for interpolation/pickle later)"
    )

    np.savez_compressed(
        OUT_NPZ,
        x=X,
        y=Y,
        wavelength_um=wl_um.astype(np.float32),
        meta_json=json.dumps(meta, indent=2)
    )
    print(f"[OK] Saved {OUT_NPZ}: X{X.shape} Y{Y.shape} wav{wl_um.shape}")
    print("NOTE:", "Cloudy run." if USE_CLOUDS else "Cloud-free run.")


if __name__ == "__main__":
    main()
